# Tool Call Without Pydantic 

In [ ]:
# Installs for the below cells.

!pip install openai
!pip install langchain
!pip install langchain-openai

In [ ]:
import os

# # Set OPENAI API Key

os.environ["OPENAI_API_KEY"] = "your openai key"

# OR (load from .env file)
# make sure you have python-dotenv installed
# from dotenv import load_dotenv
# load_dotenv("./.env")

In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.tools import tool

In [2]:
@tool
def find_name_info_from_database(name):
    """REturns information about a name from a database."""
    names_info = {
        'Alice': {'Age': 25, 'Gender': 'F'},
        'Bob': {'Age': 30, 'Gender': 'M'},
        'Charlie': {'Age': 35, 'Gender': 'M'}
    }
    
    return names_info[name]

find_name_info_from_database.invoke("Alice")

{'Age': 25, 'Gender': 'F'}

In [3]:
tools = [find_name_info_from_database]
llm = ChatOpenAI(model="gpt-4o")

llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Output(BaseModel):
    info : dict = Field(..., description="Information about the person")
    

prompt = ChatPromptTemplate.from_template("You take in a request about a person and call a function to return information about that person: {name}")

chain = prompt | llm_with_tools

tool_calls = chain.invoke({"name": "Alice"})

output = tool_calls.tool_calls[0]
selected_tool = {"find_name_info_from_database": find_name_info_from_database}[output["name"].lower()]
tool_output = selected_tool.invoke(output["args"])
tool_output

{'Age': 25, 'Gender': 'F'}

# Tool Call With Pydantic

In [6]:
from langchain_core.output_parsers.openai_tools import PydanticToolsParser
from langchain_core.pydantic_v1 import BaseModel, Field

class NameInfo(BaseModel):
    age : int = Field(..., description="Age of the person")
    status: str = Field(..., description="Gender of the person")
                      
prompt = ChatPromptTemplate.from_template("You take in a request and return info about these people: \
                                           Bob: [34, married], Mary: [25, single], Alice: [30, single],\
                                          Request: {name}.")
llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools([NameInfo])

chain = prompt | llm_with_tools | PydanticToolsParser(tools=[NameInfo])

chain.invoke({"name": "Alice"})

[NameInfo(age=30, status='single')]